## Finding the best place for a new oil well

Data on oil samples from three regions will be used to create a model that will help pick the region with the highest profit margin. The Bootstrapping technique will be used to analyze potential profit and risks.

Conditions:

- Only linear regression is suitable for model training (the rest are not sufficiently predictable).
- When exploring the region, a study of 500 points is carried with picking the best 200 points for the profit calculation.
- The budget for development of 200 oil wells is 100 USD million.
- One barrel of raw materials brings 4.5 USD of revenue The revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels).
- After the risk evaluation, keep only the regions with the risk of losses lower than 2.5%. From the ones that fit the criteria, the region with the highest average profit should be selected.

### Step 1 - Downloading and preparing the data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
df0 = pd.read_csv("/datasets/geo_data_0.csv")

In [ ]:
df1 = pd.read_csv("/datasets/geo_data_1.csv")

In [ ]:
df2 = pd.read_csv("/datasets/geo_data_2.csv")

In [ ]:
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
df1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [ ]:
df2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [ ]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [ ]:
df1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [ ]:
df2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


#### Checking for duplicates

In [ ]:
df0.loc[df0.duplicated(['id'], keep=False)]

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
1364,bxg6G,0.411645,0.856830,-3.653440,73.604260
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
16633,fiKDv,0.157341,1.028359,5.585586,95.817889
21426,Tdehs,0.829407,0.298807,-0.049563,96.035308
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205


In [ ]:
df1.loc[df1.duplicated(['id'], keep=False)]

,id,f0,f1,f2,product
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [ ]:
df2.loc[df2.duplicated(['id'], keep=False)]

,id,f0,f1,f2,product
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


#### Counting duplicate rows

In [ ]:
len(df0.loc[df0.duplicated(['id'], keep=False)].index)

20

In [ ]:
len(df1.loc[df1.duplicated(['id'], keep=False)].index)

8

In [ ]:
len(df2.loc[df2.duplicated(['id'], keep=False)].index)

8

#### Dropping duplicate rows

In [ ]:
for df in [df0, df1, df2]:
    df.drop_duplicates(subset=['id'], keep=False, inplace=True)
    df.reset_index(drop=True, inplace=True)

### Conclusion

3 files were opened and examined for general information: `geo_data_0.csv`, `geo_data_1.csv`, and `geo_data_2.csv`. 

The columns for all three datasets contain 100,000 observations and are described as follows:

`id` — unique oil well identifier

`f0`, `f1`, `f2` — three features of points (their specific meaning is unimportant, but the features themselves are significant)

`product` — volume of reserves in the oil well (thousand barrels).

All datatypes are correctly assigned, and there are 0 null values.

10 IDs in `geo_data_0.csv`, 4 IDs in `geo_data_1.csv`, and 4 IDs in `geo_data_2.csv` were found to have a duplicate. The duplicated IDs have non-matching values for `f0`, `f1`, `f2`, and `product`. Since these duplicates only comprise 0.0002, 0.00008, and 0.00008 of the dataset observations respectively, and it is not known which of the rows contain correct data, the duplicates are dropped from each dataset.

### Step 2 - Train and test the model for each region:

In [ ]:
saved_predictions_valid = []
saved_target_valid = []

for df, name in [(df0, "df0"), (df1, "df1"), (df2, "df2")]:
    target = df['product']
    features = df.drop(['id', 'product'], axis=1)

    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state = 12345)
    
    scaler = StandardScaler()
    scaler.fit(features_train)
    scaler.transform(features_train, copy=False)
    scaler.transform(features_valid, copy=False)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    predictions_valid = model.predict(features_valid)
    
    saved_predictions_valid.append(predictions_valid)
    saved_target_valid.append(target_valid)

    rmse = mean_squared_error(target_valid, predictions_valid) ** 0.5

    print('')
    print(name, "has an average volume of predicted reserves of", round(predictions_valid.mean(), 2), "thousand barrels and a RMSE of", round(rmse, 2))


df0 has an average volume of predicted reserves of 92.42 thousand barrels and a RMSE of 37.72

df1 has an average volume of predicted reserves of 68.98 thousand barrels and a RMSE of 0.89

df2 has an average volume of predicted reserves of 95.12 thousand barrels and a RMSE of 39.98


### Conclusion

The three dataframes were split using train_test_split() into a 75:25 ratio: a training dataset (75%), and a validating dataset (25%).

Numerical values were scaled using `StandardScaler` to standardize the data. Next, the model was trained as a Linear Regression.

`predictions_valid` and `target_valid` for each model was saved in `saved_predictions_valid` and `saved_target_valid` for easy retrieval.

Among the three datasets, it appears that `df0` and `df2` have similar statistics to each other than `df1`. The region represented by `df1` has the lowest RMSE, indicating better fit.

### Step 3 - Preparing for profit calculation:

In [ ]:
budget_200_wells = 100000000
num_wells = 200
revenue_1000_barrels = 4500

In [ ]:
# 0 < (x * revenue_1000_barrels) - budget_200_wells / 200

break_even_vol = budget_200_wells / num_wells / revenue_1000_barrels
print("Volume of reserves sufficient for developing a new well without losses:", round(break_even_vol, 2))

Volume of reserves sufficient for developing a new well without losses: 111.11


In [ ]:
for df, name in [(df0, "df0"), (df1, "df1"), (df2, "df2")]:
    print("Average volume of reserves in", name, ":", df['product'].mean())

Average volume of reserves in df0 : 92.49916597893447
Average volume of reserves in df1 : 68.82414772665173
Average volume of reserves in df2 : 94.99876686768079


### Conclusion

`budget_200_wells` and `revenue_1000_barrels` were created based on the initial given information.

Algebra was used to calculate the break-even point for developing a new well, which was found to be 111.11 thousand barrels. This number is higher the average volume of reserves in all three regions.

### Step 4 - Writing a function to calculate profit from a set of selected oil wells and model predictions:

In [ ]:
def profit(target, predictions):
    preds_sorted = pd.Series(predictions, index=target.index).sort_values(ascending=False)
    selected = target[preds_sorted.index][:200]
    
    return selected.sum(), selected.sum() * revenue_1000_barrels - budget_200_wells

In [ ]:
for num in range(3):
    x, y = profit(saved_target_valid[num], saved_predictions_valid[num])
    print(f"--- df{num} ---")
    print("Target reserve sum:", round(x, 2), "thousand barrels")
    print('Profit:', round(y, 2), "USD")
    print("")

--- df0 ---
Target reserve sum: 29191.17 thousand barrels
Profit: 31360260.57 USD

--- df1 ---
Target reserve sum: 27589.08 thousand barrels
Profit: 24150866.97 USD

--- df2 ---
Target reserve sum: 27702.1 thousand barrels
Profit: 24659457.92 USD



### Conclusion

A function was created to calculate profit by sorting model predictions in descending order and using their index to select the top 200 targets. Next, the targets were summed up. This value was then multiplied by the revenue from one unit of product and lastly the budget is subtracted to find the profit.

The region represented by `df0` yielded the highest profit here, due to the higher volume of oil reserves.

### Part 5 - Calculate risks and profit for each region:

In [ ]:
state = np.random.RandomState(12345)

for num in range(3):
    values = []
    for i in range(1000):
        subsample = saved_target_valid[num].sample(n=500, replace=True, random_state=state)
        preds_subsample = pd.Series(saved_predictions_valid[num], index=saved_target_valid[num].index)[subsample.index]
        target.reset_index(drop=True)
        values.append(profit(subsample, preds_subsample)[1])
    
    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)
    loss = round(values.loc[lambda x : x < 0].count() / len(values) * 100, 2)

    print(f"--- df{num} ---")
    print("Average profit:", values.mean())
    print("2.5% percentile:", lower)
    print("97.5% percentile:", upper)
    print("Risk of losses:", loss)
    print("")

--- df0 ---
Average profit: 4590856.134113576
2.5% percentile: -902340.6710362419
97.5% percentile: 9879538.83110182
Risk of losses: 4.7

--- df1 ---
Average profit: 5235793.832267862
2.5% percentile: 1337911.9122885156
97.5% percentile: 9614501.610437708
Risk of losses: 0.6

--- df2 ---
Average profit: 3414720.0526126255
2.5% percentile: -1983559.341135003
97.5% percentile: 8527085.807650132
Risk of losses: 11.6



### Conclusion

The bootstrapping technique was used with 1000 samples to find the distribution of profit. 

The region represented by `df1` is the best region for development of oil wells since it has the highest average profit of $5,235,793.83 and the lowest risk of loss of 0.6%.

Regions `df0` and `df2` exceed the risk cap requirement of 2.5%, so these two regions cannot be considered under the given criteria.